In [1]:
pip install pyspark


     -------------------------------------- 281.3/281.3 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 199.7/199.7 kB 2.0 MB/s eta 0:00:00
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: still running...
  Running setup.py install for pyspark: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install findspark


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pyspark
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [8]:
session = SparkSession.builder.appName('app').getOrCreate()
print(session)
session

In [9]:
dept=[("Finance",10),("Marketing",20),("HR",30),("Sales",50),("Support",40),("IT",60),("Admin",70),("Revenue",80)]
rdd=session.sparkContext.parallelize(dept)
print(type(dept))
print(rdd.collect())
print(rdd.take(2))
print("RDD Count:", rdd.count())
print(rdd.countByKey())
result = (rdd.filter(lambda x: x[1] > 20).collect())
print(result)
rdd1 = session.sparkContext.parallelize(result)
result2 = (rdd1.filter(lambda x: x[1] > 30).collect())
print(result2)

<class 'list'>
[('Finance', 10), ('Marketing', 20), ('HR', 30), ('Sales', 50), ('Support', 40), ('IT', 60), ('Admin', 70), ('Revenue', 80)]
[('Finance', 10), ('Marketing', 20)]
RDD Count: 8
defaultdict(<class 'int'>, {'Finance': 1, 'Marketing': 1, 'HR': 1, 'Sales': 1, 'Support': 1, 'IT': 1, 'Admin': 1, 'Revenue': 1})
[('HR', 30), ('Sales', 50), ('Support', 40), ('IT', 60), ('Admin', 70), ('Revenue', 80)]
[('Sales', 50), ('Support', 40), ('IT', 60), ('Admin', 70), ('Revenue', 80)]


In [10]:
print(rdd.getNumPartitions())
rdd3 = rdd.coalesce(2)
print(rdd3.getNumPartitions())
a=rdd3.toDF()
print(a)
a.show(truncate=False)

8
2
DataFrame[_1: string, _2: bigint]
+---------+---+
|_1       |_2 |
+---------+---+
|Finance  |10 |
|Marketing|20 |
|HR       |30 |
|Sales    |50 |
|Support  |40 |
|IT       |60 |
|Admin    |70 |
|Revenue  |80 |
+---------+---+



In [12]:
print('RDD Num Partitions:', rdd3.getNumPartitions())

RDD Num Partitions: 2


In [13]:
a = rdd.toDF()
print(a)
print(type(a))

DataFrame[_1: string, _2: bigint]
<class 'pyspark.sql.dataframe.DataFrame'>


In [19]:
a.show(truncate=False)
columns = ["Name", "id"]
a = session.createDataFrame(data=dept,schema=columns)
a.filter(a.id == 10).show(truncate=False)
a.coalesce(2)
(a.rdd.getNumPartitions())
a.repartition(8)

+---------+---+
|Name     |id |
+---------+---+
|Finance  |10 |
|Marketing|20 |
|HR       |30 |
|Sales    |50 |
|Support  |40 |
|IT       |60 |
|Admin    |70 |
|Revenue  |80 |
+---------+---+

+-------+---+
|Name   |id |
+-------+---+
|Finance|10 |
+-------+---+



DataFrame[Name: string, id: bigint]

In [22]:
data = session.read.option('header', 'true').csv("titanic.csv")
data.show()
print(type(data))

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [23]:
rdd3 = data.filter(data.Pclass == 3).show(truncate=False)

+-----------+--------+------+---------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                     |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                  |male  |22  |1    |0    |A/5 21171       |7.25   |null |S       |
|3          |1       |3     |Heikkinen, Miss. Laina                                   |female|26  |0    |0    |STON/O2. 3101282|7.925  |null |S       |
|5          |0       |3     |Allen, Mr. William Henry                                 |male  |35  |0    |0    |373450          |8.05   |null |S       |
|6          |0       |3     |Moran, Mr. James                                         |m

In [28]:
data1 = [("James","","Smith","36636","M",60000),
        ("Michael","Rose","","40288","M",70000),
        ("Robert","","Williams","42114","",400000),
        ("Maria","Anne","Jones","39192","F",500000),
        ("Jen","Mary","Brown","","F",0)]

In [29]:
data1


[('James', '', 'Smith', '36636', 'M', 60000),
 ('Michael', 'Rose', '', '40288', 'M', 70000),
 ('Robert', '', 'Williams', '42114', '', 400000),
 ('Maria', 'Anne', 'Jones', '39192', 'F', 500000),
 ('Jen', 'Mary', 'Brown', '', 'F', 0)]